# CONFIGURAÇÕES INICIAIS

»» Recursos de interesse:

- Markdown: https://www.markdownguide.org/


## Diretorias


In [1]:
#Colocar os path correto

path = "C:/Users/.../"

path_dados = "C:/Users/.../"

## Ativar os módulos de interesse

In [2]:
import requests
import json

import pickle

import math 
import pandas as pd

## Funções auxiliares

In [3]:
# A função "jprint()" tem como objetivo "imprimir" o json (formato de resposta escolhido / solicitado à API) 
# O formato ".json" é uma linguagem utilizada para representação de dados num formato do tipo "chave-valor" - numa estrutura de dados similar a um "dicionário", 
# disponibilizada na linguagem Python. 
# Os dados, tal como numa base de dados usual, podem ter uma estrutura relacional; é usual que estas relações possam ser representadas num formato
# hierárquico. O formato json é assim compatível com esta representação relacional hierárquica, sendo a navegação entre os diferentes níveis definida pelas chaves, 
# que correspondem aos nós da representação hierárquica em rede dos referidos dados codificados no formato json
# Esta função permite representar a estrutura hierárquica do json, através da "impressão" em formato textual, utilizando tabulações por forma a alinhar as chaves que se encontram
# em cada nível hierárquico

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=6)
    print(text)



## Configuração Chave API

In [4]:
MY_API_KEY = "colocar aqui a chave da API"
MY_API_KEY

'd0178a452f02df51b64ba796b0b9b4da'

# RECOLHA DE DADOS - Parte 1
Pesquisa e recolha de resultados com a API Scoupus API

## Análise exploratória dos resultados de pesquisa

### Pesquisa exploratória API

In [6]:
# IMPORTANTE: https://dev.elsevier.com/sc_search_tips.html

user_query = "data mining housing automated valuation model"
cursor = "*"

response = requests.get("https://api.elsevier.com/content/search/scopus",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY},
                         
                    params={    'query' : user_query,
                                #'count' : 1000,
                                #'start' : "25",
                                #'cursor' : '*',
                        }

                        )

print(response.url)



https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model


### Identificar número total de resultados obtidos na pesquisa

In [12]:
number_of_articles_retrieved = results["search-results"]["opensearch:totalResults"]
number_of_articles_retrieved

'256'

In [13]:
number_of_articles_perResultPage = results["search-results"]["opensearch:itemsPerPage"]
number_of_articles_perResultPage

'25'

## Implementação global - Parte 1 (DOIs)
Recolha de resultados individuais da API, seleção de dados de interesse, criação de estrutura de dados amigável

In [36]:
user_query = "data mining housing automated valuation model"
LEVEL0_FIELDS_OF_INTEREST = ['dc:title', "prism:doi", 'dc:creator']
AFFILIATION_FIELDS_OF_INTEREST = ['affilname', 'affiliation-city', 'affiliation-country']

df_results_list = pd.DataFrame(columns=LEVEL0_FIELDS_OF_INTEREST+AFFILIATION_FIELDS_OF_INTEREST)

start = 0

while start <= int(number_of_articles_retrieved):
    
    
    response = requests.get("https://api.elsevier.com/content/search/scopus",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY},
                         
                    params={    'query' : user_query,
                                'start' : start
                                #'cursor' : cursor,
                        }

                        )
    print(i)
    print(response.url)
    print(response.status_code)



    if(response.status_code == 200):

        results_aux = response.json()

        df_level1_aux = pd.DataFrame.from_records(results_aux["search-results"]['entry'] )
        df_level1_aux = df_level1_aux[LEVEL0_FIELDS_OF_INTEREST]

        if 'affiliation' in results["search-results"]['entry'][0]:
            
            df_affil_aux = pd.json_normalize(results["search-results"],['entry', 'affiliation'] )
            df_affil_aux = df_affil_aux[AFFILIATION_FIELDS_OF_INTEREST]
            df_all_aux = pd.merge(df_level1_aux, df_affil_aux, how='left', left_index=True, right_index=True)
            df_results_list = df_results_list.append(df_all_aux, ignore_index=True)

        else:

            df_affil_aux = pd.DataFrame(np.nan, index=[0],columns=AFF_COL_NAMES)
            df_all_aux = pd.merge(df_level1_aux, df_affil_aux, how='left', left_index=True, right_index=True)
            df_results_list = df_results_list.append(df_all_aux, ignore_index=True)
            
    start = start+int(number_of_articles_perResultPage)

    
    

24
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&start=0
200
24
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&start=26
200
24
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&start=52
200
24
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&start=78
200
24
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&start=104
200
24
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&start=130
200
24
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&start=156
200
24
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+automated+valuation+model&start=182
200
24
https://api.elsevier.com/content/search/scopus?query=data+mining+housing+a

In [38]:
df_results_list[:300]

,dc:title,prism:doi,dc:creator,affilname,affiliation-city,affiliation-country
0,Automated real estate valuation with machine l...,10.1016/j.eswa.2022.119147,Baur K.,Universität Freiburg,Freiburg im Breisgau,Germany
1,Predicting Carpark Prices Indices in Hong Kong...,10.32604/cmes.2022.020930,Li R.Y.M.,Hong Kong Shue Yan University,Hong Kong,China
2,House price prediction using hedonic pricing m...,10.1002/cpe.7342,Zaki J.,Faculty of Engineering,Mansoura,Egypt
3,Gridded land use data for the conterminous Uni...,10.1038/s41597-022-01591-0,Mc Shane C.,University of Colorado Boulder,Boulder,United States
4,DOES MACHINE LEARNING PREDICTION DAMPEN THE IN...,10.3846/ijspm.2022.17590,Jung J.,Hanyang University ERICA Campus,Ansan,South Korea
...,...,...,...,...,...,...
242,A new approach to spatial analysis in CAMA,10.1108/02637470510631474,González M.A.S.,Tsinghua University,Beijing,China
243,From concept towards reality: Developing the a...,10.1144/1470-9236/04-072,Culshaw M.G.,Saint Petersburg Mining University,Saint Petersburg,Russian Federation
244,Lees' Loss Prevention in the Process Industrie...,10.1016/B978-0-7506-7555-0.X5081-6,Mannan M.S.,Uniwersytet Warminsko-Mazurski w Olsztynie,Olsztyn WM,Poland
245,Quality function deployment: A literature review,10.1016/S0377-2217(02)00178-9,Chan L.K.,Gebze Teknik Üniversitesi,Kocaeli,Turkey


In [39]:
df_results_list["prism:doi"]

0              10.1016/j.eswa.2022.119147
1               10.32604/cmes.2022.020930
2                        10.1002/cpe.7342
3              10.1038/s41597-022-01591-0
4                10.3846/ijspm.2022.17590
                      ...                
242             10.1108/02637470510631474
243              10.1144/1470-9236/04-072
244    10.1016/B978-0-7506-7555-0.X5081-6
245         10.1016/S0377-2217(02)00178-9
246               10.1111/1540-6229.00048
Name: prism:doi, Length: 247, dtype: object

# RECOLHA DE DADOS - Parte 2


## Recolha de resumos (dos artigos) 

Recolha do contéudo dos resumos dos artigos selecionados com a API Scopus (usando o identificador DOI guardado anteriormente)

NOTA: Como facilmente se verifica na [página](https://dev.elsevier.com/api_docs.html) os resumos podem ser obtidos de duas formas:
* Através do serviço «ScienceDirect APIs» e especificamente da API «Article Retrieval» (documentação [aqui](https://dev.elsevier.com/documentation/ArticleRetrievalAPI.wadl) ) descrita anteriormente
* Através do serviço «Scopus APIs» (usado anteriormente para fazer a pesquisa) e especificamente da API «Abstract Retrieval» (documentação [aqui](https://dev.elsevier.com/documentation/AbstractRetrievalAPI.wadl))

## Implementação Global - Parte 2

In [41]:
dois = df_results_list['prism:doi']
dois

0              10.1016/j.eswa.2022.119147
1               10.32604/cmes.2022.020930
2                        10.1002/cpe.7342
3              10.1038/s41597-022-01591-0
4                10.3846/ijspm.2022.17590
                      ...                
242             10.1108/02637470510631474
243              10.1144/1470-9236/04-072
244    10.1016/B978-0-7506-7555-0.X5081-6
245         10.1016/S0377-2217(02)00178-9
246               10.1111/1540-6229.00048
Name: prism:doi, Length: 247, dtype: object

In [191]:
abstract_list = []

i=0
for x in dois :
    print(x)
    response_abst = requests.get("https://api.elsevier.com/content/abstract/doi/"+str(x),
                    headers={
                    'Accept': 'application/json',
                    'X-ELS-APIKey': MY_API_KEY},
                    
                    params={   }
                )
    
    print(response_abst.url)
    print(response_abst.status_code)
    print(i)
    i=i+1

    if(response_abst.status_code == 200):
        results_abst = response_abst.json()
        if 'dc:description' in results_abst["abstracts-retrieval-response"][ "coredata"]:
            abstract_list.append( results_abst["abstracts-retrieval-response"][ "coredata"]['dc:description'] )
        else:
            abstract_list.append("NULL")
    else:
        abstract_list.append("NULL")




10.3390/su132313088
https://api.elsevier.com/content/abstract/doi/10.3390/su132313088
200
0
here
10.1186/s40537-021-00476-0
https://api.elsevier.com/content/abstract/doi/10.1186/s40537-021-00476-0
200
1
here
10.1016/j.jobe.2021.102636
https://api.elsevier.com/content/abstract/doi/10.1016/j.jobe.2021.102636
200
2
here
10.1108/IJHMA-07-2020-0080
https://api.elsevier.com/content/abstract/doi/10.1108/IJHMA-07-2020-0080
200
3
here
10.3390/geosciences11110480
https://api.elsevier.com/content/abstract/doi/10.3390/geosciences11110480
200
4
here
10.1016/j.landurbplan.2021.104217
https://api.elsevier.com/content/abstract/doi/10.1016/j.landurbplan.2021.104217
200
5
here
10.1016/j.jclepro.2021.128824
https://api.elsevier.com/content/abstract/doi/10.1016/j.jclepro.2021.128824
200
6
here
10.3390/su132011450
https://api.elsevier.com/content/abstract/doi/10.3390/su132011450
200
7
here
10.3390/su132011259
https://api.elsevier.com/content/abstract/doi/10.3390/su132011259
200
8
here
10.1007/s12599-021-00

In [199]:
abstract_list[0]

'© 2021 by the authors. Licensee MDPI, Basel, Switzerland.The accurate estimation of real estate value helps the development of real estate policies that can respond to the complexities and instability of the real estate market. Previously, statistical methods were used to estimate real estate value, but machine learning methods have gained popularity because their predictions are more accurate. In contrast to existing studies that use various machine learning methods to estimate the transactions or list prices of real estate properties without separating the building and land prices, this study estimates land price using a large amount of land-use information obtained from various land-and building-related datasets. The random forest and XGBoost methods were used to estimate 52,900 land prices in Seoul, South Korea, from January 2017 to December 2020. The models were also separately trained for different land uses and different time periods. Overall, the results revealed that XGBoost 

In [194]:
type(abstract_list[0])

str

# GUARDAR DADOS RECOLHIDOS E PROCESSADOS

In [201]:
df_results_list.to_csv(path_dados+'search_results.csv',  index=False, encoding = "utf-8")

pd.DataFrame(abstract_list,  columns =['Abstract']).to_csv(path_dados+'search_results_abstracts.csv', index=False, encoding = "utf-8")

In [ ]:
# Save the file
#pickle.dump( df_results_list, file = open(path_dados+"ICD2122_TextMining_DataStructures.pickle", "wb"))

# Reload the file
#test_grouped_df_reloaded = pickle.load(open(path_dados+"ICD2122_TextMining_DataStructures.pickle", "rb"))